# <center style="font-family: consolas; font-size: 32px; font-weight: bold;"> ♟️ UM - Game-Playing Strength of MCTS Variants - 📊 Understanding MCTS Variants</center>
<p><center style="color:#949494; font-family: consolas; font-size: 20px;">Predict the performance of MCTS variants in board games</center></p>

***

In this notebook we will take a look at the general MCTS algorithm, and study the different variations that have been tried to simulate and generate the dataset in this competition!

Hope you enjoy ❤️

# Table of Contents

* [0. Introduction](#0)
* [1. Monte-Carlo Tree Search](#1)
* [2. Selection](#2)
* [3. Exploration Constant](#3)
* [4. Playout](#4)
* [5. Score Bounds](#5)


<a id="0"></a>
# 0. Introduction

The competition organizers have tried various types of Monte-Carlo Tree Search variants, lets understand what each of these are. 

From the [data page](https://www.kaggle.com/competitions/um-game-playing-strength-of-mcts-variants/data):

> All agent string descriptions in training and test data are in the following format: `MCTS-<SELECTION>-<EXPLORATION_CONST>-<PLAYOUT>-<SCORE_BOUNDS>`, where:
>
> - `<SELECTION>` is one of: `UCB1`, `UCB1GRAVE`, `ProgressiveHistory`, `UCB1Tuned`. These are different strategies that may be used within the Selection phase of the MCTS algorithm.
> - `<EXPLORATION_CONST>` is one of: `0.1`, `0.6`, `1.41421356237`. These are three different values that we have tested for the "exploration constant" (a numeric hyperparameter shared among all of the tested Selection strategies).
> - `<PLAYOUT>` is one of: `Random200`, `MAST`, `NST`. These are different strategies that may be used within the Play-out phase of the MCTS algorithm.
> - `<SCORE_BOUNDS>` is one of: `true` or `false`, indicating whether or not a "Score-Bounded" version of MCTS (a version of the algorithm that can prove when certain nodes in its search tree are wins/losses/draws under perfect play, and adapt the search accordingly).
>
> For example, an MCTS agent that uses the `UCB1GRAVE` selection strategy, an exploration constant of `0.1`, the `NST` play-out strategy, and Score Bounds, will be described as `MCTS-UCB1GRAVE-0.1-NST-true`.
> 
> You may treat every distinct agent string as a completely separate agent, but you may also try to leverage the compositional nature of the MCTS agents and split it up into its components.

Let's first take a look at the MCTS algorithm and then delve into what these variants are.

<a id="1"></a>
# 1. Monte-Carlo Tree Search

Monte-Carlo Tree Search is a **heuristic search** algorithm commonly used to solve a game tree. Unlike exact search methods, heuristic search methods use heuristic functions, simpler functions that estimate the utility of exploring specific paths to reach a goal, approximating exact solutions, usually having to be employed in problems that are NP-complete or NP-hard. 

It is based on the Monte-Carlo method, and is a Tree Search algorithm. Let's understand what these mean:
- **Monte-Carlo**: Monte-Carlo methods rely on continual random sampling under a distribution to solve problems, with the idea that through various random experiments we can find concrete solutions.
- **Tree Search**: the algorithm solves tree-like problems, where they can be described as starting from an initial state (the root of the tree) and making decisions along discrete time, building the branches of the tree. A solution is simply a path that goes from the root to a leaf node. Usually, the complete tree is intractable to store or traverse, so instead of looking at a global image of the problem, the algorithms that carry out tree search include some form of locality.

The MCTS algorithm consists of iteratively computing four steps until you reach a solution: Selection, Expansion, Simulation and Backpropagation. Here, we will build a tree with scores at each node to denote how good of a state each node is for our agent:
1. **Selection**: randomly select a path from the root (the current game state) to a leaf (any state with children whose score is yet unknown)
2. **Expansion**: if the leaf is not decisive (i.e. doesn't end the game), choose any child via a valid move of the game
3. **Simulation**: continue playing by choosing random moves until the game ends
4. **Backpropagation**: use the result of the game to update the scores of the nodes that compose the path


<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F5838132%2Ffb7db1c47d649a902f2c34d7c5510a70%2F2024-09-07_14-24.png?generation=1725711885686769&alt=media" alt="MCTS algorithm"></center> 
<center>MCTS algorithm steps. Source: <a href="https://dke.maastrichtuniversity.nl/m.winands/documents/CG2010mp.pdf">https://dke.maastrichtuniversity.nl/m.winands/documents/CG2010mp.pdf</a></center>

<a id="2"></a>
# 2. Selection

The selection step is the most important one as choosing better or worse leaf nodes (i.e. those closer to or further from optimal play) may guide the model to explore better or worse paths. Additionally, there is a question of whether it is better to explore paths that go further into the tree, or consider new ones with leaf nodes that are closer to the root and are yet unexplored. Exploration vs exploitation that must also be taken into a account.

The competition organizers mention that their agents employ one of the following selection methods: UCB1, UCB1GRAVE, ProgressiveHistory and UCB1Tuned. What do these entail?

## UCB1

Presented in [Finite-time Analysis of the Multiarmed Bandit Problem](https://homes.di.unimi.it/~cesabian/Pubblicazioni/ml-02.pdf), it is an Upper Confidence Bounds algorithm. Given a choice $j$, an average reward of the choice $X_j$, with $n_j$ being the number of times it is taken and $n$ being the total decisions made in a game, UCB1 does the following:
1. Make each choice once (to obtain a starting reward and have an idea of which choices may be better than others)
2. Play the machine $j$ that maximizes:

$$X_j + \sqrt{\frac{C \ln n}{n_j}}$$

We include a constant $C$ to determine the exploration factor, that is, how much more to favor exploring new paths over exploiting existing ones. Note the following interesting observation: if two choices with the same average reward exist, the algorithm will opt for the one that has been chosen the least (given the $n_j$ in the denominator).

A good resource to undertsand where this formula originates from can be [this Jeremy Kun blogpost](https://www.jeremykun.com/2013/10/28/optimism-in-the-face-of-uncertainty-the-ucb1-algorithm/).

However, it is important to note that the UCB1 algorithm adapted to tree search modifies this into a new algorithm, UCT (Upper Confidence applied to Trees), where given an average reward $X_j$ of child $j$, UCT maximizes:

$$X_j + C \times \sqrt{\frac{\ln n_p}{n_j}}$$

Here instead of considering the total plays $n$, we locally consider the number of times a parent $p$ has been visited only.

## UCB1Tuned

It is a variation of UCB1, also proposed in the same paper as UCB1, where the authors modify the square root term that also considers the variance of the rewards of each choice (thus being *tuned*) and performs better in all of the paper's experiments. Specifically, the maximization is carried out on the following expression:

$$X_j + \sqrt{\frac{\ln n}{n} \min\{1/4, V_j(n_j)\}}$$

with $V_j(s)$ being:

$$V_j(s) = \big(\frac{1}{s} \sum^s_{\tau=1} X^2_{j,\tau}\bigg) - \bar{X}^2_{j, s} + \sqrt{\frac{2 \ln t}{s}}$$

where $s$ is the number of times a machine has been played, $t$ denotes the time step. Here we see that $V_j(s)$ essentially calculates a difference between the sample variance of the rewards and the variance of the average rewards. So, UCB1Tuned employs additional information about how the average rewards are changing over time, which makes it a tune to specific games with specific reward functions.

Again, note that this is the general form and usually for MCTS, people tune the UCT variant instead.

## UCB1GRAVE

Introduced in [Generalized Rapid Action Value Estimation](https://www.ijcai.org/Proceedings/15/Papers/112.pdf), it combines ideas from UCB1 and RAVE, Rapid Action Value Estimation, which enhances the exploratory phase of the MCTS algorithm by storing more information as the tree is traversed.

Specifically, in RAVE, for a given game tree node $N$, its child nodes $C_i$ store not only the statistics of wins in playouts started in node $N$ but also the statistics of wins in *all playouts* started in node $N$ and below it, if they contain move $i$ (also when the move was played in the tree, between node $N$ and a playout). This way **the contents of tree nodes are influenced not only by moves played immediately in a given position but also by the same moves played later**..


<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e8/Tic-tac-toe-RAVE-English.svg/441px-Tic-tac-toe-RAVE-English.svg.png" alt="RAVE"></center> 
<center>RAVE applied to Tic-Tac-Toe.</center>

UCB1GRAVE, however, uses a generalized version of RAVE that, instead of considering the statistics of wins in all playouts, it considers only those with at least $k$ wins, Of course, if set to $k=0$, GRAVE would become RAVE.
Mathematically, this means that instead of adding the square root term to the average reward, another term is added. The paper describes this in much more detail, please take a look!

## ProgressiveHistory

It is presented in [Enhancements for Multi-Player Monte-Carlo Tree Search](https://dke.maastrichtuniversity.nl/m.winands/documents/CG2010mp.pdf) as a way to include more historical knowledge into the update equations that must be maximized in the selection step of MCTS. Specifically, the child $j$ with the highest score is chosen as:

$$X_j + C \times \sqrt{\frac{\ln n_p}{n_j}} + X^{win}_a \times \frac{W}{n_j - s_j + 1}$$

Here the first two terms correspond to the UCT formula, while the last one adds information about the score across the history of the game. here, $W$ is a constant parameter that defines how much the history will affect the choice, $s_j$ is the sum of rewards obtained by choosing child $j$, and $X^{win}_a$ represents the average reward when carrying out move $a$. 

<a id="3"></a>
# 3. Exploration Constant

This one is pretty simple. In the previous section we saw that the way we select the next child is by looking at which one has the highest score. The formula for the highest score varies depending on the selection strategy that is chosen, but all of them include the $C$ constant to balance between how much to explore new paths, or exploit existing ones.

The competition organizers employed `0.1`, `0.6`, and `1.41421356237` as three possible values for the exploration constants. From the formulas, we can easily observe that the larger this constant becomes, the more these algorithms will favor exploration over exploitation. Importantly, for ProgressiveHistory, the $C$ constant does not have as pronounced an effect as in the other algorithms, given the added effect of including historical knowledge into the maximization.

<a id="4"></a>
# 4. Playout

Once we have selected a path, chosen the next move (or moves) to make, the next step of the MCTS algorithm consists in playing the game until we reach an end state. It is important to design the playout step in an intelligent manner to correctly estimate the score of the chosen children and slowly move toward making more optimal choices in future selection steps. 

The competition organizers employ three playout strategies: Random200, MAST and NST. Let's understand what these are!

## Random200

This refers to considering the distribution of all possible moves $a$ in a given state $s$ to be uniformly distributed, and randomly picking one until either the game ends or 200 steps are realized (in which case the score of the path is assigned some undisclosed default value). Hence, here the probability $P(a)$ that an action $a$ will be chosen is:

$$P(a) = \frac{1}{|A|}$$

## MAST

[Simulation-Based Approach to General Game Playing](https://cdn.aaai.org/AAAI/2008/AAAI08-041.pdf) introduces what they later call [the MAST search-control method](https://www.cs.huji.ac.il/~jeff/aaai10/02/AAAI10-169.pdf), winner of the General Game Playing Competition at AAAI in 2009. MAST, Moving-Average Sampling Technique, refers to the strategy in which actions are chosen to get to new states with unkown values by also considering historical information. They do this with the following principle:

- Actions that are good in one state are often also good in other states.

This is known as a history heuristic. So, instead of choosing from an uniformly-distributed set of actions, the distribution of actions to choose from becomes dependent on the historical observations of action values independently of the state in which they had been played. Essentially, they use existing $Q$ values $Q(s, a)$\*, but averaging over $s$ to get the mean action value $Q(a)$ (this is where the "moving-average" comes from in MAST's name), and then employ a softmax with temperature $\tau$ to get the new action distribution to sample from:

$$P(a) = \frac{e^{Q(a) / \tau}}{\sum_{b\in A} e^{Q(b) / \tau}}$$

\*If you're unaware of what $Q(s, a)$ is, [this is a good starting point](https://datascience.stackexchange.com/questions/9832/what-is-the-q-function-and-what-is-the-v-function-in-reinforcement-learning).

## NST

The same authors of MAST went on to generalize their strategy with NST, the  N-Gram Selection Technique in [N-Grams and the Last-Good-Reply Policy applied in General Game Playing](https://project.dke.maastrichtuniversity.nl/games/files/articles/TCAIG_Ngram.pdf). Here, instead of considering the values of single actions, they look at sequences of $n$ actions, with $n=1$ making NST = MAST. While this can be costly given that now instead of considering $|A|$ values we consider $|A|^n$, by increasing $n$ to $n=2$ there are already [improvements in games such as Havannah](https://www.mendeley.com/catalogue/418890b3-2641-3de8-8bd8-f881ad10f923/).

Simply, the idea is to calculate $P(a_t)$, the probability of taking action $a_t$ at time-step $t$, by also considering the $n-1$ actions at previous time-steps (in practice we only use windows of 1 or 2 previous actions). For this, as is commonly done in language or other sequential domains, we usually apply a [Markov assumption](https://machinelearninginterview.com/topics/natural-language-processing/what-order-of-markov-assumption-is-done-for-n-grams/):

$$P(a_t \mid a_{t-1}, a_{t-2}, \ldots, a_0) \approx P(a_t \mid a_{t-1}, \ldots, a_{t-n+1})$$

<a id="6"></a>
# 5.Score Bounds

In [Score Bounded Monte-Carlo Tree Search](https://cgi.cse.unsw.edu.au/~abdallahs/Papers/2010/Score%20Bounded%20Monte-Carlo%20Tree%20Search.pdf) the authors propose a modified MCTS algorithm that can prove the result of choosing a specific node and following its path under perfect play, allowing it to adapt and improve the search accordingly.

Specifically, for each state $s$ they calculate pessimistic and optimistic bounds on its score, $pess(s)$ and $opti(s)$. As more information becomes available, $pess(s)$ increases and $opti(s)$ decreases, both sandwiching the real value of $s$. If the state is an end state, $pess(s) = opti(s) = score(s)$, otherwise 

$$pess(s) = \text{opt}_{c\in children(s)} pess(c)$$ 

and  

$$opti(s) = \text{opt}_{c\in children(s)} opti(c)$$

where $\min$ or $\max$ are denoted by $\text{opt}$ depending on whether the MCTS agent is a MAX or MIN agent.

Once having obtained the bounds for the nodes, they are propagated in a specific way to the top:

<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F5838132%2Fffd7189eefd79ca4b1e102b630cbbedb%2F2024-09-07_17-20.png?generation=1725722437457904&alt=media" alt="Propagating pessimistic bounds"></center> 
<center>Propagating pessimistic bounds.</center>

<center><img src="https://www.googleapis.com/download/storage/v1/b/kaggle-forum-message-attachments/o/inbox%2F5838132%2Fbd8dbaa403c4d7b2bc3e6dcc2392a691%2F2024-09-07_17-21.png?generation=1725722470126866&alt=media" alt="Propagating pessimistic bounds"></center> 
<center>Propagating optimistic bounds.</center>


Then, provided that we are situated in a state $s$, we can use simple rules to prune nodes similarly to [$\alpha\beta$ algorithms.](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) and reduce the search space: if $opti(c) \leq pess(s)$ do not explore child $c$, or if $pess(c) \geq opti(s)$ do not explore child $c$.

# Thanks

I hope this notebook was informative to you! Full disclosure, I was keen on learning more about MCTS algorithms and did not have an incentive to do so until now, so I took the opportunity to dive a bit deeper into it and in turn made this notebook 😁

I appreciate any feedback you may have and hope you can use this to get more context about the competition and maybe even some insight on any new features that may be useful.